# Tutorial 6: Galactic Binaries & RJMCMC

In the sixth tutorial, we will examine Galactic Binary waveforms. We will then use them in fixed-dimensional MCMC and then in RJMCMC. We use RJMCMC to perform model selection on the number of sources in the data. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from lisatools.utils.constants import *
from copy import deepcopy  # can be useful
from lisatools.utils.constants import *
from lisatools.sensitivity import get_sensitivity

## Task 1: Build and plot a Galacic Binary waveform using `GBGPU`

We will start by generating Galactic Binary waveforms with `GBGPU`. Pick reasonable parameters, build a waveform and plot it against the LISA A channel TDI Sensitivity Curve (`A1TDISens`) in the characteristic strain representation. You can access the information after waveform generation as attributes on the class. This may be updated in the future.

Useful documentation:
* [GBGPU](https://mikekatz04.github.io/GBGPU/html/user/main.html#gbgpu.gbgpu.GBGPU)

In [ ]:
# imports
from gbgpu.gbgpu import GBGPU

In [ ]:
gb = GBGPU()

## Task 2: Run an MCMC over a single GB source

Run a fixed-dimensional MCMC run with a chosen GB source. Fix the sky location for now to simplify the problem computationally (this is especially important for the next section on RJ with GBs). So, you will sample over 6 of the 8 parameters. Discuss or think about reasonable priors for these parameters and how you would determine that. For simplicity, we recommend using tightly (but not too tightly) bound uniform distributions for this example setup.

There is a faster `get_ll` method on the `GBGPU` class. However, it may be easier to use the full `AnalysisContainer` setup. This will make the RJ part more straight forward, but is not actually ideal for fixed-dimensional MCMC on GBs. 

AFter the run is complete, plot the posterior distribution with `chainconsumer` or `corner`. 

In [ ]:
from eryn.prior import uniform_dist, ProbDistContainer
from lisatools.analysiscontainer import AnalysisContainer
from lisatools.datacontainer import DataResidualArray
from lisatools.sensitivity import AE1SensitivityMatrix
from eryn.ensemble import EnsembleSampler
from eryn.state import State

## Task 3: RJ with GBs

Our final task will be to run RJMCMC on a few close Galactic Binaries. The key component here is the "global" Likelihood function. Work to build a function that takes from Eryn and adjustable length array of templates to be summed into a global template prior to the Likelihood computations. This will be a bit tedious, but is very important for understanding this process. 

You will also have to introduce a `GaussianMove` like we did in the last lesson. We recommend picking a simple setup for the purposes of the tutorial. There are better ways to do this that would take more time (e.g. information matrix -> covariance calculation). 

If you can run the sampler and confirm the Likelihoods are working, then consider this completed. The time alloted for the tutorial and the overall setup needed to run this RJ setup correclty require a lot more runtime for reasonable results. So, you can plot what comes out, but it will become more accurate as your run the sampler longer. 

In [ ]:
def fill_template(template, A, E, freqs, Tobs, dt):
    for i in range(A.shape[0]):
        start_ind = gb.start_inds[i]
        end_ind = start_ind + N_wave
        
        assert end_ind - start_ind == gb.freqs.shape[1]
        template[0, start_ind:end_ind] += A[i]
        template[1, start_ind:end_ind] += E[i]

def generate_global_template(template, params_all, Tobs, dt):
    gb.run_wave(*params_all.T, T=Tobs, dt=dt, N=N_wave)  # T=Tobs, oversample=4)
    fill_template(template, gb.A, gb.E, gb.freqs, Tobs, dt)
    
def global_log_likelihood(params_all, analysis, Tobs, dt, default_values):
    input_parameters = np.zeros((params_all.shape[0], 9))
    input_parameters[:, np.array([0, 1, 2, 4, 5, 6])] = params_all
    input_parameters[:, np.array([7, 8])] = default_values
    # print(input_parameters)
    
    template = np.zeros_like(analysis.data_res_arr[:])

    generate_global_template(template, input_parameters, Tobs, dt)
    template_in = DataResidualArray(template, f_arr=f_arr)
    ll = analysis.template_likelihood(template_in)
    return ll

In [ ]:
injection_params_all = priors["gb"].rvs(size=5)
input_parameters = np.zeros((injection_params_all.shape[0], 9))
input_parameters[:, np.array([0, 1, 2, 4, 5, 6])] = injection_params_all
input_parameters[:, np.array([7, 8])] = default_values
data = np.zeros((2, len(f_arr)), dtype=complex)

generate_global_template(data, input_parameters, Tobs, dt)
inds = np.where(data[0])
plt.loglog(f_arr[inds], np.abs(data[0][inds]))